In [ ]:
from dotenv import *
import os, subprocess
load_dotenv(find_dotenv())

project_root = '..'

In [ ]:
import librosa, scipy

source_query_path = os.path.join(project_root, os.environ['SOURCE_QUERY_PATH'])
tmp_file = 'query.ogg'
query_path = os.path.join(project_root, os.environ['TARGET_QUERY_PATH'])
commands = ['cp %s %s' % (source_query_path, tmp_file),
           'opusdec --force-wav query.ogg - | sox - -r 8k --norm -b 8 --channels 1 %s trim 0 10' % (query_path),
           'rm %s' % tmp_file]

os.system('; '.join(commands))

TODO: Remove this, we need the upload portion to work before we can actually use a query

In [ ]:
query_path = 'query.wav'

# Play the query

In [ ]:
import IPython
IPython.display.Audio(query_path)

# Chromagram of Query

In [ ]:
import librosa, numpy as np
y, sr = librosa.load(query_path)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
chroma = librosa.feature.chroma_stft(S=np.abs(librosa.stft(y)))
librosa.display.specshow(chroma, y_axis='chroma', x_axis='time')
plt.colorbar()
plt.title('Chromagram')
plt.tight_layout()

# Transcribe query

In [ ]:
from utils import get_music21_obj
song_obj = get_music21_obj(query_path)
print ' '.join(n.name for n in song_obj.flat.notes)

# Play transcribed song as midi

In [ ]:
from utils import stream_to_midi_wav
query_midi_wav_path = stream_to_midi_wav(song_obj)
IPython.display.Audio(query_midi_wav_path)

# Find 20 candidate songs

In [ ]:
%%time
from utils import get_prediction_df
res_df = get_prediction_df(query_path)

In [ ]:
res_df.head(15)

# Load the top 5 predicted songs

In [ ]:
top_n = 5
for i in range(top_n):
    s_filename = list(res_df.iloc[[i]].wav_paths)[0]
    s_error = list(res_df.iloc[[i]].error)[0]
    print '%s - Error: %s' % (s_filename, s_error)
    IPython.display.display(IPython.display.Audio(s_filename))